In [ ]:

%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.widgets import TextBox

# Set up the figure and axes
fig, (ax_left, ax_right) = plt.subplots(1, 2, figsize=(12, 6))
plt.subplots_adjust(bottom=0.3)

# Hide axes
for ax in [ax_left, ax_right]:
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_frame_on(False)

# Square parameters
square_size = 2
half = square_size / 2
corner_snap_radius = 0.3

# Global state
t1 = 0
t2 = 0
dragging_p1 = False
dragging_p2 = False
dragging_dot = False
initial_angle = None
initial_t = None
textbox1_focused = False
textbox2_focused = False

# Fixed square S1 centered at origin
s1_center = (0, 0)
s1 = patches.Rectangle((s1_center[0] - half, s1_center[1] - half),
                       square_size, square_size, fill=False, edgecolor='black', linewidth=2)
ax_left.add_patch(s1)

# Movable squares S2 and S3
s2 = patches.Rectangle((0, 0), square_size, square_size, fill=False, edgecolor='black', linewidth=2)
s3 = patches.Rectangle((0, 0), square_size, square_size, fill=False, edgecolor='blue', linewidth=2)
ax_left.add_patch(s2)
ax_left.add_patch(s3)

# Gluing point markers
p1_marker, = ax_left.plot([], [], 'ro')
p2_marker, = ax_left.plot([], [], 'bo')

# Reoriented coordinate map for torus grid
def remap_x(x):
    return {0:2, 1:3, 2:0, 3:1}[int(x) % 4] + (x % 1)

def unmap_x(x):
    return {2:0, 3:1, 0:2, 1:3}[int(x) % 4] + (x % 1)

# Configuration grid (4x4 torus)
for i in range(5):
    ax_right.plot([0, 4], [i, i], 'k-', lw=0.5)
    ax_right.plot([i, i], [0, 4], 'k-', lw=0.5)

# Red dot in configuration space
config_dot, = ax_right.plot([], [], 'ro', markersize=8)

# Compute boundary point on a square
def boundary_point(t, center):
    t_mod = t % 4
    if 0 <= t_mod < 1:
        x = center[0] + half
        y = center[1] - half + square_size * (t_mod % 1)
    elif 1 <= t_mod < 2:
        x = center[0] + half - square_size * (t_mod % 1)
        y = center[1] + half
    elif 2 <= t_mod < 3:
        x = center[0] - half
        y = center[1] + half - square_size * (t_mod % 1)
    else:
        x = center[0] - half + square_size * (t_mod % 1)
        y = center[1] - half
    return x, y

# Get square coordinates
def get_square_bounds(square):
    x, y = square.get_xy()
    return x, x + square_size, y, y + square_size

# Check if point is in square
def inside_square(x, y, square):
    xmin, xmax, ymin, ymax = get_square_bounds(square)
    return xmin <= x <= xmax and ymin <= y <= ymax

def inside_config_dot(x, y):
    dot_x, dot_y = config_dot.get_data()
    if not dot_x or not dot_y:
        return False
    return np.hypot(x - dot_x[0], y - dot_y[0]) < 0.15

# Update all square positions
def update_all():
    global t1, t2
    p1 = boundary_point(t1, s1_center)
    p2_rel = boundary_point((t1 + 2) % 4, (0, 0))
    s2_center = (p1[0] - p2_rel[0], p1[1] - p2_rel[1])
    s2.set_xy((s2_center[0] - half, s2_center[1] - half))
    p1_marker.set_data([p1[0]], [p1[1]])

    p2 = boundary_point(t2, s2_center)
    p3_rel = boundary_point((t2 + 2) % 4, (0, 0))
    s3_center = (p2[0] - p3_rel[0], p2[1] - p3_rel[1])
    s3.set_xy((s3_center[0] - half, s3_center[1] - half))
    p2_marker.set_data([p2[0]], [p2[1]])

    config_dot.set_data([remap_x(t1 % 4)], [t2 % 4])
    textbox1.set_val(f"{t1:.4f}")
    textbox2.set_val(f"{t2:.4f}")
    fig.canvas.draw_idle()

# Event handlers
def on_press(event):
    global dragging_p1, dragging_p2, dragging_dot, initial_angle, initial_t
    if event.xdata is None or event.ydata is None:
        return
    if event.inaxes == ax_left:
        if inside_square(event.xdata, event.ydata, s2):
            dragging_p1 = True
            initial_angle = np.arctan2(event.ydata - s1_center[1], event.xdata - s1_center[0])
            initial_t = t1
        elif inside_square(event.xdata, event.ydata, s3):
            s2_center = (s2.get_x() + half, s2.get_y() + half)
            dragging_p2 = True
            initial_angle = np.arctan2(event.ydata - s2_center[1], event.xdata - s2_center[0])
            initial_t = t2
    elif event.inaxes == ax_right and inside_config_dot(event.xdata, event.ydata):
        dragging_dot = True

def on_release(event):
    global dragging_p1, dragging_p2, dragging_dot
    dragging_p1 = False
    dragging_p2 = False
    dragging_dot = False

def on_motion(event):
    global t1, t2
    if event.xdata is None or event.ydata is None:
        return
    if event.inaxes == ax_left:
        if dragging_p1 or dragging_p2:
            center = s1_center if dragging_p1 else (s2.get_x() + half, s2.get_y() + half)
            angle = np.arctan2(event.ydata - center[1], event.xdata - center[0])
            if angle < 0:
                angle += 2 * np.pi
            delta_angle = angle - initial_angle
            if delta_angle > np.pi:
                delta_angle -= 2 * np.pi
            elif delta_angle < -np.pi:
                delta_angle += 2 * np.pi
            delta_t = (delta_angle / (2 * np.pi)) * 4
            new_t = (initial_t + delta_t) % 4
            if dragging_p1:
                t1 = new_t
            else:
                t2 = new_t
            update_all()
    elif event.inaxes == ax_right and dragging_dot:
        tx, ty = unmap_x(event.xdata), event.ydata
        t1 = tx % 4
        t2 = ty % 4
        update_all()

def submit_t1(text):
    global t1
    try:
        t1 = float(text) % 4
        update_all()
    except:
        pass

def submit_t2(text):
    global t2
    try:
        t2 = float(text) % 4
        update_all()
    except:
        pass

def on_key(event):
    global textbox1_focused, textbox2_focused
    if textbox1_focused and event.key in ['backspace', 'delete', 'escape']:
        textbox1.set_val("")
    if textbox2_focused and event.key in ['backspace', 'delete', 'escape']:
        textbox2.set_val("")

def on_focus(event):
    global textbox1_focused, textbox2_focused
    if event.inaxes == axbox1:
        textbox1_focused = True
        textbox2_focused = False
    elif event.inaxes == axbox2:
        textbox2_focused = True
        textbox1_focused = False
    else:
        textbox1_focused = False
        textbox2_focused = False

axbox1 = plt.axes([0.25, 0.15, 0.2, 0.05])
textbox1 = TextBox(axbox1, 't1:', initial="0.0000")
textbox1.on_submit(submit_t1)

axbox2 = plt.axes([0.55, 0.15, 0.2, 0.05])
textbox2 = TextBox(axbox2, 't2:', initial="0.0000")
textbox2.on_submit(submit_t2)

fig.canvas.mpl_connect('button_press_event', on_press)
fig.canvas.mpl_connect('button_release_event', on_release)
fig.canvas.mpl_connect('motion_notify_event', on_motion)
fig.canvas.mpl_connect('button_press_event', on_focus)
fig.canvas.mpl_connect('key_press_event', on_key)

update_all()

ax_left.set_xlim(-6, 6)
ax_left.set_ylim(-6, 6)
ax_left.set_aspect('equal')
ax_left.set_title("Three Squares Drag Configuration")

ax_right.set_xlim(-0.5, 4.5)
ax_right.set_ylim(-0.5, 4.5)
ax_right.set_aspect('equal')
ax_right.set_title("Configuration Grid $(\mathbb{R}/4\mathbb{Z})^2$")

plt.show()
